# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [212]:
%matplotlib inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

In [213]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [214]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    #data = open(filename, 'rb')
    #of = data.read()
    #of = bytes(pickle.load(data))
    #data.close()
    #data = data.read()
    #data.seek(0)
    #f = bytes(data.read())
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [215]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print('total number of data', len(all_data))


ValueError: could not convert string to float

In [ ]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [ ]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [ ]:
# YOUR CODE HERE
t_data = []
t_target = []
for i in training_data:
    t_data.append(all_data[i])
    t_target.append(all_target[i])
clf.fit(t_data, t_target)#[:n_training_data], t_target[:n_training_data])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (15,) + inhomogeneous part.

### predicting

In [ ]:
clf.predict(all_data[-1:]), all_target[-1:]

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [ ]:
expected = []
predicted = []
# YOUR CODE HERE
expected = t_target
predicted = clf.predict(t_data)
evaluate(expected, predicted)

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## 4. Evaluate on the test data

In [ ]:
expected = []
predicted = []
# YOUR CODE HERE
indices_to_delete = training_data

test_data = np.asarray(all_data)

test_data = np.delete(test_data, indices_to_delete, axis=0) # delete the 155 training_data_elements
# new shape: (67,10)

test_target = np.asarray(all_target)
test_target = np.delete(test_target, indices_to_delete, axis=0)

expected = test_target
predicted = clf.predict(test_data)

evaluate(expected, predicted)

AttributeError: 'function' object has no attribute 'predict'

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [ ]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [ ]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1:]

AttributeError: 'function' object has no attribute 'predict'